In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt

def creer_dossier_si_non_existant(chemin):
    if not os.path.exists(chemin):
        os.makedirs(chemin)

def charger_donnees(fichier, delimiter=';'):
    try:
        return pd.read_csv(fichier, delimiter=delimiter)
    except FileNotFoundError:
        print(f"Erreur : Le fichier {fichier} est introuvable.")
        return None

def tronquer_donnees(df, axe_temps):
    dernier_index_non_nan = df.apply(pd.Series.last_valid_index).min()
    axe_temps_tronque = axe_temps[:dernier_index_non_nan + 1]
    df_tronque = df.iloc[:dernier_index_non_nan + 1, :]

    # Assurer que les longueurs correspondent
    df_tronque = df_tronque[:len(axe_temps_tronque)]
    return df_tronque, axe_temps_tronque

def convertir_colonnes_float(df):
    for col in df.columns:
        if df[col].dtype == 'object':
            df[col] = df[col].str.replace(',', '.').astype(float)
        elif df[col].dtype == 'float64' and df[col].isnull().any():
            df[col] = df[col].astype(float)
    return df

def tracer_courbes(essai, df, axe_temps, chemin_figures):
    for col in df.columns:
        plt.plot(axe_temps, df[col], label=col)

    plt.title(f"Essai {essai}")
    plt.xlabel("Temps")
    plt.ylabel("Valeurs")
    plt.legend(loc='upper right', bbox_to_anchor=(1.3, 1.0))
    plt.grid(True)
    nom_figure = os.path.join(chemin_figures, f"ESSAI_{str(essai).zfill(3)}.png")
    plt.savefig(nom_figure)
    plt.close()

def main():
    chemin_repertoire = "Output"
    chemin_figures = os.path.join(chemin_repertoire, "figures")
    creer_dossier_si_non_existant(chemin_figures)

    FICHIER_AXE_TEMPS = "Axe_du_temps/ESSAI_1_3_6.csv"
    FICHIER_REF_DATA2 = "ACQUISITION FORGEAGE/ESSAI_114.csv"

    df_axe_temps = charger_donnees(FICHIER_AXE_TEMPS)
    if df_axe_temps is None:
        return
    axe_temps = df_axe_temps['tps']
    axe_temps.to_csv('Axe_du_temps/AXE_TEMPS.csv', index=False)

    plages_essais = [(26, 27), (31, 37), (41, 47), (51, 57), (61, 67), (71, 77), (81, 87), (91, 97), (101, 107), (111, 117), (121, 127)]
    fichiers_non_coherents = []

    for plage in plages_essais:
        debut, fin = plage
        for essai in range(debut, fin + 1):
            if essai == 113:
                continue
            nom_fichier = os.path.join("ACQUISITION FORGEAGE", f"ESSAI_{str(essai).zfill(3)}.csv")
            print(nom_fichier)
            print(os.getcwd())

            if os.path.exists(nom_fichier):
                print('exist')
                df_essai = charger_donnees(nom_fichier)
                if df_essai is not None:
                    df_essai, axe_temps_tronque = tronquer_donnees(df_essai, axe_temps)
                    df_essai = convertir_colonnes_float(df_essai)
                    tracer_courbes(essai, df_essai, axe_temps_tronque, chemin_figures)
            else:
                fichiers_non_coherents.append(nom_fichier)

    print("Les graphiques ont été générés avec succès.")
    if fichiers_non_coherents:
        print("Les fichiers suivants n'ont pas été trouvés :")
        for fichier in fichiers_non_coherents:
            print(fichier)

if __name__ == "__main__":
    main()
